# OCR : 

### L'objectif de ce notebook est de tester le package 'easyOCR' pour la detection du texte

# ----------------------------------------

    - Input : 
            + Excel file : id of scans with their label ( Motifrecla )
            + .tif file ( scans )
    
    - Output : 
            + Excel file : {Id , number of frames, text )

In [6]:
# Check cuda
import torch
print(torch.cuda.is_available()) # False : pas de gpu

False


In [7]:
# check python version used in the notebook:
from platform import python_version
print(python_version())

3.6.8


In [8]:
ls # check files

Data/                      Output_Summarization/
Data_TopicClassification/  Picke_AnalyseVerbatimSante.ipynb
Labellisation/             README.md
OCR.ipynb                  Summarization.ipynb
Output_OCR/


In [9]:
# Get path:
path ='Data/' # specify path

In [10]:
# Lib :
import os
import pandas as pd

In [11]:
fnames = os.listdir(path) # check tif files
fnames[:5]

['{BB96C6DA-5B0B-46FA-BC81-C05E59D75A36}.tif',
 '{BADB7E7E-0563-4CA0-8E25-072EBC633604}.tif',
 '{BB2D2BE9-81DA-41D8-9CC0-D77FE7AAD12E}.tif',
 '{BB153D96-F25C-40C5-8EE5-8DA14039FBFE}.tif',
 '{BAE1B1FE-2346-4A33-A623-E70617CABFC0}.tif']

In [12]:
# read data :
# Classification_emails : df contenant l'id de chaque scan et son label (motifrecla)
df = pd.read_excel(path + 'Classification_emails.xlsx')

In [13]:
df.head() # check data

,IDDEMANDE,IDDOC,TYPEDEMANDE,NUMERO_CONTRAT,BRANCHE_CONTRAT,CONTRATINDIV,GARANTIE,MOTIFRECLA,REPONSERECLA
0,11295941,{AF07E04C-73CE-48A5-B276-60BA692B3FB3},Résiliation assuré,0JY4850,M,0JY4850,NaN,Manque de proactivité,COMPL_Favorable au client
1,11308265,{6C8BBF86-2975-4FD0-8008-0124AC16310D},Demande diverse - Presta Prev,0644491,P,0644491,INCAPACITE,Délai trop long,COMPL_Favorable au client
2,11323834,{7048823A-B7DB-4455-A695-D473B235097D},Demande diverse - AFFIL,0NT1218,M,0NT1218,NaN,Délai trop long,Défavorable au client
3,11309899,{767C5A35-657E-4430-9643-9C626B935F6B},Demande diverse - Presta Prev,2254335,P,2254335,CONGES EXCEPTIONNELS,Charge administrative,COMPL_Favorable au client
4,11311700,{2B3C2EFA-C6AE-4B99-A501-DA288AD1ABD9},Demande diverse - Presta Prev,0598865,P,0598865,INCAPACITE,Délai trop long,Défavorable au client


In [19]:
# check df :
print("check shape of data : ", df.shape) # shape
print("**********")
print("")
print("**********")
print("get info of df :  ", df.info()) # info

check shape of data :  (2609, 9)
**********

**********
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2609 entries, 0 to 2608
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   IDDEMANDE        2609 non-null   int64 
 1   IDDOC            2609 non-null   object
 2   TYPEDEMANDE      2609 non-null   object
 3   NUMERO_CONTRAT   2581 non-null   object
 4   BRANCHE_CONTRAT  2608 non-null   object
 5   CONTRATINDIV     2581 non-null   object
 6   GARANTIE         1767 non-null   object
 7   MOTIFRECLA       2609 non-null   object
 8   REPONSERECLA     2609 non-null   object
dtypes: int64(1), object(8)
memory usage: 183.6+ KB
get info of df :   None


In [20]:
# check freq of MOTIFRECLA
df.MOTIFRECLA.value_counts()

Délai trop long                         1643
Manque de fiabilité                      283
Charge administrative                    175
Absence de réponse directe au client     120
Manque de proactivité                    114
Complexité de langage                    102
Mauvaise qualité du contact               68
Incohérence de prix                       61
Problématiques web                        29
Difficulté à joindre le groupe            14
Name: MOTIFRECLA, dtype: int64

### Packages OCR:

In [21]:
from easyocr import Reader 
import cv2
# import pytesseract
import shutil
try:
 from PIL import Image
except ImportError:
 import Image

In [20]:
# create DF to store scan value of .tiff with id of frames in same file

def convert_multiple_tiff_to_singular_tiffs(dataframe_photos):
    ''' From tiff containing several frames to frame per tiff
    
        Args:
            - dataframe_photos : pd.DataFrame
                                 df with column 'IDDOC' : id of scans
        
        Returns:
            - create new .tif files if any have more than 1 frame (pages)
        
        Example:
            - {00C4AF8E-D0FA-4457-84AA-B9FE0BFA2E18}.tif contain 4 frames (pages)
             the function output will be adding these files in the data folder:
                 {00C4AF8E-D0FA-4457-84AA-B9FE0BFA2E18}2.tif
                 {00C4AF8E-D0FA-4457-84AA-B9FE0BFA2E18}3.tif
                 {00C4AF8E-D0FA-4457-84AA-B9FE0BFA2E18}4.tif
    '''
    k = 0 # To keep count of which tif is being handled
    for photo in dataframe_photos.IDDOC : # Loop over all .tif files
        print(" ** ** photo id = ", k)
        k+=1
        filePath = photo + '.tif' # also name of the file = {XXXXXXX-XXXX-XXX-XXX} + '.tif'
        img = Image.open(path + filePath)
        for frame in range (1,img.n_frames): # Loop of the number of frames per tif to create the new tif files
            try:
                img.seek(frame) # Select frame/page = frame 
                img.save(path + photo + str(frame) + '.tif') # Save new file ( tif file ), same name with page number : {XXXXXXX-XXXX-XXX-XXX}1 + '.tif'
            except:
              pass

In [ ]:
convert_multiple_tiff_to_singular_tiffs(df) # get text from scans

In [21]:
# create DF to store scan value of .tiff with id of frames in same file

def parse_tif(dataframe_photos):
    ''' Parse tif files and get text
    
        Args : 
            - dataframe_photos : pd.DataFrame
                                 df with column 'IDDOC' : id of scans
        
        Returns:
            - df_scans : pd.DataFrame 
                         dataframe containing text
    
    '''
   
    languages = ['fr'] # Specify parameters - language - for the reader function
    reader = Reader(languages, gpu = True) # We don't have GPU in our case - could change it to False
    filePath_list = [] # list to store names of photo
    frames_list = [] # list to store number of the frame of the photo
    text_list = [] # list to store scan of each frame
    compt = 0 # to check script in case of error
    
    for photo in dataframe_photos.IDDOC : # Loop over tif files (only the originals)
        text = "" # empty text
        print(" ** ** photo id = ", compt) # printing to check advancement of the script
        compt+=1
        
        filePath = photo + '.tif' # Path of the file and also its name
        img = Image.open(path + filePath) # needed to get number of frames
        filePath_list.append(filePath) # append list
        frames_list.append(img.n_frames) # append list
        
        for i in range (0, img.n_frames): # Loop over the frames of each of the original tif
            sub_text = "" # initialize sub text
            try:
                # first iteration :
                if i == 0:
                    print(" PASSED HERE ===================================")
                    IMAGE_PATH = path + photo + '.tif'
                else:
                    IMAGE_PATH = (path + photo + str(i) + '.tif')
                    
                image = cv2.imread(IMAGE_PATH) # read tif (image) file
                results = reader.readtext(image) # Store output of ocerization
                
                for item in results:
                    #a = a + ' ' + item[1]
                    sub_text = sub_text + '\n' + item[1] # change format of output

                text = text + sub_text # Sum all text of the frames
            except:
              pass
        
        if compt<4: # just checking outputs
            print(text)
            
        text_list.append(text) # Append list of outputs/results
            
    df_scans = pd.DataFrame({"tiff_name": filePath_list, "frames": frames_list, "text": text_list }) # Convert to DF
    
    return df_scans

In [22]:
df_out = parse_tif(df)

CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.


 ** ** photo id =  0
 PASSED HERE ===================================


CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.



Page 1
From:
"BG DLSP Affiliations"
affiliations@ag2rlamondiale fr>
Date:
30 Dec 2021 17.41:10 +0100
To:
"BG
MAILS
SP
AFFIL_NUMERISATION'
<BG
MAILS_SP
AFFIL_NUMERISATION@agZrlamondiale fr>
Subject:
TR
[5997594-1640762236] 28/12/2021
1561211076279
CALMETTES DIDIER
Attachments:
28/12/2021
1561211076279
CALMETTES DIDIER
De
Didier CALMETTES <dfcalmettes@gmail.com>
Envoyé
jeudi 30 décembre 2021 17.40.58 (UTC+01.00) Brussels; Copenhagen; Madrid, Paris
À
BG CG BORDEAUX AFFILIATIONS; BG DL SP Affiliations
Sujet
[5997594-1640762236] 28/12/2021
1561211076279
CALMETTES DIDIER
AG2R LA MONDIALE
Commentaires de 1'Agent Hind TPFVA ZELMAT BAGHDAD
Merci de prendre ce dossier en charge.
Information sur le client
Numéro Interne
10608808
Prénom
Didier
Nom
CALMETTES
Email
df calmettes@gmail.com
Description de la requête
Attention; mail de gestion technique: Ce mail contient des données personnelles client et des données de santé.
Bonjour;
Vous trouverez ci-dessous les éléments afin de traiter la demande d

CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.


...

 ** ** photo id =  2606
 PASSED HERE ===================================


CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.


 ** ** photo id =  2607
 PASSED HERE ===================================


CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.


 ** ** photo id =  2608
 PASSED HERE ===================================


In [23]:
df_out.shape

(2609, 3)

In [24]:
df_out.head(1)

,tiff_name,frames,text
0,{AF07E04C-73CE-48A5-B276-60BA692B3FB3}.tif,3,"\nPage 1\nFrom:\n""BG DLSP Affiliations""\naffil..."


In [25]:
df_out.to_excel('output_esyocr.xlsx', index=False) # Download output